In [1]:
import os 
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
api_key = os.getenv("GOOGLE_API_KEY")
if api_key is None:
    raise ValueError("key not found")

model = ChatGoogleGenerativeAI(model='gemini-1.5-flash')

In [3]:
class BlogState(TypedDict):

    title: str
    outline: str
    blog: str
    content: str

In [4]:
def create_outline(state: BlogState) -> BlogState:

    # fetch title
    title = state['title']

    # call llm gen outline
    prompt = f"Create a detailed outline for a blog post with the title: {title}"
    outline = model.invoke(prompt).content

    # update state
    state['outline'] = outline

    return state

In [5]:
def create_blog(state: BlogState) -> BlogState:
    
    title = state['title']
    outline = state['outline']

    prompt = f"Write a detailed blog post based on the title: {title} and the outline: {outline}"

    content = model.invoke(prompt).content

    state['content'] = content

    return state

In [6]:
graph = StateGraph(BlogState)

# nodes
graph.add_node('create_outline',create_outline)
graph.add_node('create_blog',create_blog)

# edges
graph.add_edge(START, 'create_outline')
graph.add_edge('create_outline', 'create_blog')
graph.add_edge('create_blog', END)  

# compile
workflow = graph.compile()


In [7]:
initial_state = {'title': "The rise of AI in modern technology"}
final_state = workflow.invoke(initial_state)
print(final_state['outline'])

## Blog Post Outline: The Rise of AI in Modern Technology

**I. Introduction (approx. 100 words)**

* **Hook:** Start with a compelling anecdote or statistic showcasing AI's impact in daily life (e.g., personalized recommendations, self-driving cars, medical diagnoses).
* **Brief definition of AI:**  Explain Artificial Intelligence in simple terms, avoiding jargon. Focus on its ability to mimic human intelligence.
* **Thesis statement:** State the main argument – that AI is rapidly transforming modern technology across various sectors and its impact is only accelerating.
* **Roadmap:** Briefly outline the sections to be covered in the post.


**II.  Key Areas of AI Development and Application (approx. 300-400 words)**

* **A. Machine Learning (ML):**
    * Explain ML's core concept (learning from data without explicit programming).
    * Examples:  Image recognition (facial recognition, medical imaging), natural language processing (chatbots, language translation), predictive analytics

In [9]:
print(final_state['content'])

## The Rise of AI in Modern Technology

**I. Introduction**

Have you ever received a perfectly tailored product recommendation online, just moments after browsing a similar item?  Or perhaps you've interacted with a chatbot that seamlessly answered your questions? These are just a few examples of artificial intelligence (AI) subtly shaping our daily lives.  Artificial intelligence, in simple terms, refers to the development of computer systems able to perform tasks that typically require human intelligence, such as learning, reasoning, and problem-solving. This blog post will explore the rapid rise of AI in modern technology, examining its key areas of development, its transformative impact across various industries, and the ethical considerations that accompany this powerful technology. We'll also delve into the future of AI and its potential to reshape our world.


**II. Key Areas of AI Development and Application**

AI's rapid advancement is driven by several key areas of developme